In [6]:
import pandas as pd
from google.cloud import bigquery
from prettytable import PrettyTable

Set up GBQ for Both John's and Natalie's Wedge Project

In [9]:
# Authenticate and set up the first client
gbq_project_ID_John = "UMT-MSBA"
gbq_dataset_ID_John = "wedge_transactions"
client1 = bigquery.Client(project = gbq_project_ID_John)


gbq_project_ID_Nat = "wedge-project-np"
gbq_dataset_ID_Nat = "wedge"
client2 = bigquery.Client(project = gbq_project_ID_Nat)


## Total Rows

In [12]:

# Example SQL queries for each client
query1 = """
SELECT count(*) as cnt
FROM `umt-msba.wedge_transactions.transArchive_*`
-- January/Oct 2012 Rows
SELECT EXTRACT(Year from datetime) as Yr,
EXTRACT(Month from datetime) as Mo,
count(*)
FROM `umt-msba.wedge_transactions.transArchive_*`
GROUP BY Yr, Mo
HAVING Yr = 2012
ORDER BY Yr, Mo
-- January = ?
-- October =
"""
query2 = """
SELECT count(*) as cnt
FROM `wedge-project-np.wedge.transArchive_*`
-- January/Oct 2012 Rows
SELECT EXTRACT(Year from datetime) as Yr,
EXTRACT(Month from datetime) as Mo,
count(*)
FROM `wedge-project-np.wedge.transArchive_*`
GROUP BY Yr, Mo
HAVING Yr = 2012
ORDER BY Yr, Mo
-- January = ?
-- October =
"""

# Run the queries
result1 = client1.query(query1).to_dataframe()
result2 = client2.query(query2).to_dataframe()

# Print results or compare them
print(result1)
print(result2)

BadRequest: 400 POST https://bigquery.googleapis.com/bigquery/v2/projects/UMT-MSBA/jobs?prettyPrint=false: ProjectId must be non-empty

Location: None
Job ID: 328e22f5-a8d4-4ccf-83c6-1e85a8ee1104


In [14]:
gbq_project_ID_John = "UMT-MSBA"
gbq_dataset_ID_John = "wedge_transactions"
client1 = bigquery.Client(project = gbq_project_ID_John)

# Example SQL queries for each client
query1 = """
SELECT EXTRACT(Year FROM datetime) AS Yr,
       EXTRACT(Month FROM datetime) AS Mo,
       COUNT(*) AS cnt
FROM `umt-msba.wedge_transactions.transArchive_*`
WHERE EXTRACT(Year FROM datetime) = 2012
  AND EXTRACT(Month FROM datetime) IN (1, 10)  -- Filter for January and October
GROUP BY Yr, Mo
ORDER BY Yr, Mo;
"""


# Run the queries
result1 = client1.query(query1).to_dataframe()


# Print results or compare them
print(result1)


BadRequest: 400 POST https://bigquery.googleapis.com/bigquery/v2/projects/UMT-MSBA/jobs?prettyPrint=false: ProjectId must be non-empty

Location: None
Job ID: 2cd188f7-b9a2-4aee-900e-671b597a06b4


In [ ]:
SELECT count(*) as cnt
FROM `umt-msba.wedge_transactions.transArchive_*`
--FROM `your_project.your_dataset.your_table_prefix_*`
-- January/Oct 2012 Rows
SELECT EXTRACT(Year from datetime) as Yr,
EXTRACT(Month from datetime) as Mo,
count(*)
FROM `umt-msba.wedge_transactions.transArchive_*`
--FROM `your_project.your_dataset.your_table_prefix_*`
GROUP BY Yr, Mo
HAVING Yr = 2012
ORDER BY Yr, Mo
-- January = ?
-- October =

In [ ]:
- Rows by Month
SELECT EXTRACT(Month from datetime) as Mo,
count(*) as cnt
FROM `umt-msba.wedge_transactions.transArchive_*`
--FROM `your_project.your_dataset.your_table_prefix_*`
WHERE EXTRACT(Year from datetime) <= 2016
GROUP BY Mo
ORDER BY cnt
-- January = ?
-- October = 

In [ ]:
- Null Counts
SELECT COUNTIF(trans_subtype is NULL) as Null_TS,
COUNTIF(datetime is NULL) as Null_DT,
COUNTIF(local IS NULL) as Null_Local,
COUNTIF(card_no IS NULL) as Null_CN
FROM `umt-msba.wedge_transactions.transArchive_*`
--FROM `your_project.your_dataset.your_table_prefix_*`
-- Each value

In [ ]:
-- High volume cards
SELECT card_no,
COUNT(*) as cnt
FROM `umt-msba.wedge_transactions.transArchive_*`
--FROM `your_project.your_dataset.your_table_prefix_*`
WHERE card_no != 3
GROUP BY card_no
ORDER BY cnt desc
LIMIT 10
-- What card is #5 on the list and how many rows does it have?
-- 18736 Rows
SELECT card_no,
COUNT(*) as cnt
FROM `umt-msba.wedge_transactions.transArchive_*`
--FROM `your_project.your_dataset.your_table_prefix_*`
WHERE card_no = 18736
GROUP BY card_no
-- How many rows do I hav

In [ ]:
-- Popular Products
SELECT LOWER(Description),
COUNT(*) as cnt
FROM `umt-msba.wedge_transactions.transArchive_*`
--FROM `your_project.your_dataset.your_table_prefix_*`
WHERE trans_type = "I" AND
Description != "Discount"
GROUP BY LOWER(Description)
ORDER BY cnt desc
LIMIT 10
-- Which product is #1? How many rows does it have?
-- Which product is #4? How many rows does it have?

In [ ]:
Single-record items
SELECT
COUNT(DISTINCT Description) as SingleRecordItems
FROM (
SELECT LOWER(Description) as Description,
COUNT(*) as cnt
FROM `umt-msba.wedge_transactions.transArchive_*`
--FROM `your_project.your_dataset.your_table_prefix_*`
WHERE trans_type = "I" AND Description != "Discount"
GROUP BY Description
HAVING cnt = 1
)

In [ ]:
Owner Fractions by Year
SELECT
Y,
OwnerRows,
NonOwnerRows,
ROUND(OwnerRows/(OwnerRows+NonOwnerRows),4) AS OwnerFrac
FROM (
SELECT
EXTRACT(Year
FROM
datetime) AS Y,
COUNTIF(card_no != 3) AS OwnerRows,
COUNTIF(card_no = 3) AS NonOwnerRows
FROM
`umt-msba.wedge_transactions.transArchive_*`
--`your_project.your_dataset.your_table_prefix_*`
GROUP BY
Y)
ORDER BY
Y
-- Which year has the highest portion of owner rows? What year? What fraction?
-- Which year has the lowest portion of owner rows? What year? What fraction?


In [ ]:
from prettytable import PrettyTable

# Create a PrettyTable object
table = PrettyTable()

# Define the columns
table.field_names = ['Query', 'Your Results', 'John\'s Results', 'Difference', 'Rel. Diff']

# Add rows with your results
table.add_row(['Total Rows', your_total_rows, john_total_rows, diff_total_rows, rel_diff_total_rows])
table.add_row(['January 2012 Rows', your_jan_2012_rows, john_jan_2012_rows, diff_jan_2012_rows, rel_diff_jan_2012_rows])
table.add_row(['October 2012 Rows', your_oct_2012_rows, john_oct_2012_rows, diff_oct_2012_rows, rel_diff_oct_2012_rows])
table.add_row(['Month with Fewest', your_month_fewest, john_month_fewest, 'Yes/No', 'NA'])
table.add_row(['Num Rows in Month with Fewest', your_rows_month_fewest, john_rows_month_fewest, diff_rows_month_fewest, rel_diff_rows_month_fewest])
# Add more rows as needed

# Set alignment for better readability (optional)
table.align = 'l'

# Print the table
print(table)